In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
# Definir a dimensão do ruído de entrada para a RNG
latent_dim = 100

In [ ]:
# Carregar o conjunto de dados MNIST
(x_train, _), (_, _) = mnist.load_data()

In [ ]:
# Construir o gerador
generator = Sequential()
generator.add(Dense(7 * 7 * 128, input_dim=latent_dim))
generator.add(Reshape((7, 7, 128)))
generator.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', activation='relu'))
generator.add(Conv2DTranspose(1, kernel_size=4, strides=2, padding='same', activation='tanh'))

# Construir o discriminador
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=(28, 28, 1), activation='relu'))
discriminator.add(Conv2D(128, kernel_size=4, strides=2, padding='same', activation='relu'))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

# Compilar o discriminador
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

# Congelar os pesos do discriminador durante o treinamento do gerador
discriminator.trainable = False

# Construir a RNG
gan = Sequential()
gan.add(generator)
gan.add(discriminator)

In [ ]:
# Compilar a RNG
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

In [ ]:
# Função para amostrar ruído aleatório para a entrada do gerador
def generate_latent_points(latent_dim, n_samples):
    return np.random.randn(latent_dim * n_samples).reshape((n_samples, latent_dim))

# Função para gerar imagens usando o gerador
def generate_images(generator, latent_dim, n_samples):
    noise = generate_latent_points(latent_dim, n_samples)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Normalizar as imagens para o intervalo [0, 1]
    return generated_images

# Função para exibir as imagens geradas
def plot_generated_images(images):
    fig, axs = plt.subplots(5, 5)
    count = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(images[count, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            count += 1
    plt.show()

# Treinamento da RNG
epochs = 50
batch_size = 128
n_samples = 2

In [ ]:
# Treinamento da RNG (continuação)
for epoch in range(epochs):
    for batch in range(x_train.shape[0] // batch_size):
        # Treinar o discriminador
        real_images = x_train[batch * batch_size: (batch + 1) * batch_size]
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))
        noise = generate_latent_points(latent_dim, batch_size)
        generated_images = generator.predict(noise)

        discriminator_loss_real = discriminator.train_on_batch(real_images, real_labels)
        discriminator_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
        discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

        # Treinar o gerador
        noise = generate_latent_points(latent_dim, batch_size)
        generator_loss = gan.train_on_batch(noise, real_labels)

    # Imprimir métricas de treinamento a cada época
    print(f'Epoch {epoch+1}/{epochs} - Discriminator Loss: {discriminator_loss[0]:.4f} - '
          f'Discriminator Accuracy: {discriminator_loss[1]*100:.2f}% - Generator Loss: {generator_loss:.4f}')

    # Gerar imagens para visualização
    if (epoch + 1) % 10 == 0:
        generated_images = generate_images(generator, latent_dim, 25)
        plot_generated_images(generated_images)
